In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib as plt
from tqdm import tqdm, trange
import torch
import pickle

In [2]:
#set parameters
vertices = 1000
adjacencies = 100
pi_vals = [0.25, 0.5, 0.75]
# pi_vals = [0.25]
num_trials = 1000

In [3]:
replicate = {}

In [4]:
device = torch.device("cuda")

In [5]:
# c is treatment assignment
# beta 1 is treatment assignment
# beta 2 is first degree neighbors
# beta 3 is second degree neighbors
def add_test_to_dictionary(beta_1, beta_2, beta_3, replicate):
  for pi in pi_vals:
    ADE = []
    AIE_1 = []
    AIE_2 = []
    AIE_3 = []
    AIE_4 = []
    for i in trange(1000):
      G = nx.random_regular_graph(adjacencies, vertices)

      # first degree neighbor matrix
      A = nx.adjacency_matrix(G).todense()

      # second degree neighbor matrix
      A_gpu = torch.tensor(A).float().to(device)
      A_2 = torch.matmul(A_gpu, A_gpu).cpu().numpy()
      A_2 = np.where(A_2 > 0, 1, 0)
      np.fill_diagonal(A_2, 0)
      A_2 = A_2 - A # second degree neighbors
      A_2_gpu = torch.tensor(A_2).float().to(device)

      # treatment assignment
      W = np.random.binomial(1, pi, vertices)
      W_gpu = torch.tensor(W).float().to(device)

      # noise
      noise = np.random.normal(0, 1, vertices)

      # generate Y
      # no interference
      if beta_2 == 0 and beta_3 == 0:
        ei = beta_1
      # only first degree neighbor
      elif beta_3 == 0:
        ei = beta_1 + beta_2 * torch.matmul(W_gpu, A_gpu).cpu().numpy() / adjacencies
      # only second degree neighbor
      elif beta_2 == 0:
        ei = beta_1 + beta_3 * torch.matmul(W_gpu, A_2_gpu).cpu().numpy() / np.sum(A_2, axis=1)
      # first and second degree neighbor
      else:
        ei = beta_1 + beta_2 * (torch.matmul(W_gpu, A_gpu).cpu().numpy() / adjacencies) + beta_3 * (torch.matmul(W_gpu, A_2_gpu).cpu().numpy() / np.sum(A_2, axis=1))

      Y = W * (ei - 3 * (ei - 1/2)**3) + noise

      # ADE
      average_direct_effect = ((np.dot(Y,W))/(pi) - (np.dot(Y, (1-W)))/(1-pi))/vertices
      ADE.append(average_direct_effect)

      # AIE
      # E = first degree
      average_indirect_effect1 = np.dot(Y, ((W @ A)/pi - ((1 - W) @ A) / (1 - pi))) / vertices
      average_indirect_effect2 = np.dot(Y, ((W @ A_2)/pi - ((1 - W) @ A_2) / (1 - pi))) / vertices
      AIE_1.append(average_indirect_effect1)
      AIE_2.append(average_indirect_effect2)

    str1 = str(pi)+" ADE_{}_{}_{}".format(round(beta_1, 2), round(beta_2, 2), round(beta_3, 2))
    str2 = str(pi)+" AIE_1_{}_{}_{}".format(round(beta_1, 2), round(beta_2, 2), round(beta_3, 2))
    str3 = str(pi)+" AIE_2_{}_{}_{}".format(round(beta_1, 2), round(beta_2, 2), round(beta_3, 2))
    replicate[str1] = ADE
    replicate[str2] = AIE_1
    replicate[str3] = AIE_2

    with open('12_9_23_Equation_3_RandomGraph.pkl', 'wb') as fp:
      pickle.dump(replicate, fp)


In [6]:
# betas = []
# # baseline where no interference
# betas.append((0, 0))

# # only first degree neighbors
# betas.append((1, 0))

# # first and second degree neighbors
# betas.append((1/3, 2/3))
# betas.append((1/2, 1/2))
# betas.append((2/3, 1/3))
betas = []
# baseline where no interference
betas.append((1, 0, 0))

#only first degree neighbors
betas.append((1/3, 2/3, 0))
betas.append((0.5, 0.5, 0))
betas.append((2/3, 1/3, 0))

#only second degree neighbors
betas.append((1/3, 0, 2/3))
betas.append((0.5, 0, 0.5))
betas.append((2/3, 0, 1/3))

#first and second degree neighbors
betas.append((1/3, 5/9, 1/9))
betas.append((1/3, 1/3, 1/3))
betas.append((1/3, 1/9, 5/9))

In [7]:
# only second degree neighbors
# betas.append((0, 1))

In [8]:
for beta in betas:
  add_test_to_dictionary(beta[0], beta[1], beta[2], replicate)

100%|██████████| 1000/1000 [06:32<00:00,  2.55it/s]


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [11]:
df_r = pd.DataFrame.from_dict(replicate)
df_r.head()

,0.25 ADE_1_0_0,0.25 AIE_1_1_0_0,0.25 AIE_2_1_0_0,0.5 ADE_1_0_0,0.5 AIE_1_1_0_0,0.5 AIE_2_1_0_0,0.75 ADE_1_0_0,0.75 AIE_1_1_0_0,0.75 AIE_2_1_0_0,0.25 ADE_0.33_0.67_0,...,0.75 AIE_2_0.33_0.33_0.33,0.25 ADE_0.33_0.11_0.56,0.25 AIE_1_0.33_0.11_0.56,0.25 AIE_2_0.33_0.11_0.56,0.5 ADE_0.33_0.11_0.56,0.5 AIE_1_0.33_0.11_0.56,0.5 AIE_2_0.33_0.11_0.56,0.75 ADE_0.33_0.11_0.56,0.75 AIE_1_0.33_0.11_0.56,0.75 AIE_2_0.33_0.11_0.56
0,0.552963,-1.857717,-14.420746,0.617042,-2.244340,-35.336123,0.651289,0.438545,7.131093,0.526281,...,59.852300,0.433345,0.818011,2.179563,0.582965,-1.265818,-18.050623,0.889017,4.851056,45.665956
1,0.539104,-1.502570,-11.514266,0.646524,0.605233,8.716451,0.697028,4.415982,50.638726,0.338884,...,38.381571,0.373882,-0.227857,-9.388096,0.621847,-0.922994,-7.373951,0.606058,-0.112427,4.207550
2,0.533537,0.065754,-4.722723,0.590465,1.010246,7.610403,0.559378,1.650886,16.274178,0.520037,...,-43.203276,0.446802,-0.190860,-3.707525,0.721812,5.009098,43.723572,0.731814,2.471902,27.989796
3,0.744268,3.693491,15.852623,0.589714,3.776771,28.363747,0.591980,-0.053779,-5.469009,0.472389,...,-81.354237,0.469170,-0.290556,-4.544535,0.657364,2.197509,23.343628,0.774868,1.302568,15.512472
4,0.522834,-1.305107,-9.930573,0.613948,-2.160853,-30.842415,0.633509,1.001157,14.682375,0.552364,...,-48.312867,0.555189,-0.711558,-0.547800,0.674938,0.693237,3.372618,0.761944,5.039490,34.573007


In [12]:
df_r.mean()

0.25 ADE_1_0_0               0.626753
0.25 AIE_1_1_0_0             0.086533
0.25 AIE_2_1_0_0            -0.133794
0.5 ADE_1_0_0                0.623514
0.5 AIE_1_1_0_0              0.089110
                               ...   
0.5 AIE_1_0.33_0.11_0.56     0.038887
0.5 AIE_2_0.33_0.11_0.56     0.338562
0.75 ADE_0.33_0.11_0.56      0.725957
0.75 AIE_1_0.33_0.11_0.56   -0.125028
0.75 AIE_2_0.33_0.11_0.56   -0.809492
Length: 90, dtype: float64

In [13]:
df_r.to_csv('12_9_23_Equation_3_RandomGraph.csv')